# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!Powershell.exe -Command type publications.tsv -Head 3

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2024-09-11	Interpreting the spatial distribution of soil properties with a physically-based distributed hydrological model	Geoderma Regional	Open access!	"Libohova, Z., M. Mancini, H. E. Winzeler, Q. D. Read, N. Sun, D. Beaudette, C. Williams, J. Blackstock, S. H. G. Silva, N. Curi, K. Adhikari, A. Ashworth, J. Minia, and P. R. Owens. 2024. Interpreting the spatial distribution of soil properties with a physically-based distributed hydrological model. Geoderma Regional: e00863. DOI: 10.1016/j.geodrs.2024.e00863."	libohova-et-al-2024	https://doi.org/10.1016/j.geodrs.2024.e00863
2024-08-12	A natural polymer material as a pesticide adjuvant for mitigating off-target drift	Heliyon	Open access!	"Kannan, N., Q. D. Read, and W. Zhang. A natural polymer material as a pesticide adjuvant for mitigating off-target drift. 2024. Heliyon 10(16): e35510. DOI: 10.1016/j.heliyon.2024.e35510."	kannan-et-al-2024	https://doi.org/10.1016/j.heliyon.20

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications.head()


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2024-09-11,Interpreting the spatial distribution of soil ...,Geoderma Regional,Open access!,"Libohova, Z., M. Mancini, H. E. Winzeler, Q. D...",libohova-et-al-2024,https://doi.org/10.1016/j.geodrs.2024.e00863
1,2024-08-12,A natural polymer material as a pesticide adju...,Heliyon,Open access!,"Kannan, N., Q. D. Read, and W. Zhang. A natura...",kannan-et-al-2024,https://doi.org/10.1016/j.heliyon.2024.e35510
2,2024-08-03,Effects of full-fat high-oleic soybean meal in...,Poultry Science,Open access!,"Toomer, O. T., P. Maharjan, K. L. Harding, T. ...",toomer-et-al-2024,https://doi.org/10.1016/j.psj.2024.104074
3,2024-07-05,Environmental influences on drying rate of spr...,PhytoFrontiers,Open access!,"Copes, W. E., Q. D. Read, and B. J. Smith. Env...",copes-et-al-2024,https://doi.org/10.1094/PHYTOFR-03-24-0019-R
4,2024-07-01,Human milk oligosaccharides and Bifidobacteria...,Current Developments in Nutrition,NaN,"Mulakala, B. K., M. Salinas, J. Rearick, B. On...",mulakala-et-al-2024,https://doi.org/10.1016/j.cdnut.2024.103429


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [5]:
# Here, subset the rows of publications to only 1 or however many we need to update right now
publications = publications.head(1)

In [6]:
# Some test code to see what the filename would be
str(publications['pub_date'])

'0    2024-09-11\nName: pub_date, dtype: object'

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [7]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = str(item.pub_date)[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    ##md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [8]:
%%cmd
dir /s /o:-d ..\_publications\ 

Microsoft Windows [Version 10.0.22631.3880]
(c) Microsoft Corporation. All rights reserved.

C:\Users\Quentin.Read\Documents\GitHub\qdread.github.io\markdown_generator>dir /s /o:-d ..\_publications\ 
 Volume in drive C is OS
 Volume Serial Number is 262B-7618

 Directory of C:\Users\Quentin.Read\Documents\GitHub\qdread.github.io\_publications

08/09/2024  12:34 PM               749 2024-08-03-toomer-et-al-2024.md
08/09/2024  12:34 PM    <DIR>          .
07/24/2024  11:06 AM               666 2024-07-05-copes-et-al-2024.md
07/24/2024  11:06 AM               781 2024-07-01-mulakala-et-al-2024.md
06/19/2024  06:12 AM    <DIR>          ..
05/27/2024  09:10 AM               694 2024-05-27-owens-et-al-2024.md
05/27/2024  08:56 AM               794 2024-02-16-rering-et-al-2024.md
05/24/2024  02:31 PM               767 2024-03-26-gurung-et-al-2024.md
05/24/2024  02:31 PM               691 2024-02-06-mengistu-et-al-2024.md
05/24/2024  02:31 PM               824 2024-05-06-elkins-et-al-2024.md
0